In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import load_model
from keras.datasets import mnist
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocess the data
X_train = X_train.astype(np.float64) / 255.0
X_test = X_test.astype(np.float64) / 255.0

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

# Build the model
model = keras.Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dropout(0.25))

model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=4, verbose=1)
ms = ModelCheckpoint('./best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True)

callbacks = [es, ms]

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_split=0.3, callbacks=callbacks)

# Save the model
model.save('digit_model.h5')

# Load the model
loaded_model = load_model('digit_model.h5')

# Import required libraries for drawing in Jupyter Notebook
from IPython.display import display, clear_output
import ipywidgets as widgets

# Set up widgets for drawing and displaying predictions
canvas = widgets.Output(layout={'border': '1px solid black'})
prediction_label = widgets.Label()


model.evaluate(X_test,y_test)
from sklearn import metrics
preds = np.round(model.predict(X_test),0)
labels = ['0','1','2','3','4','5','6','7','8','9']

classification_metrics = metrics.classification_report(y_test,preds,target_names=labels)
print(classification_metrics)




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.datasets import mnist

# Load the MNIST dataset
(_, _), (X_test, _) = mnist.load_data()

# Preprocess the data
X_test = X_test.astype(np.float64) / 255.0
X_test = np.expand_dims(X_test, -1)

# Load the model
model = load_model('/content/digit_model.h5')

# Define class labels
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# Choose an example index from the test dataset
example_index = 0  # You can change this index to view different examples

# Preprocess the image for prediction
example_img = X_test[example_index]
preprocessed_img = np.expand_dims(example_img, axis=0)

# Make prediction
proba = model.predict(preprocessed_img)
top_3 = np.argsort(proba[0])[:-4:-1]
for i in range(3):
    print("{} ({:.3f})".format(labels[top_3[i]], proba[0][top_3[i]]))

# Display the example image
plt.imshow(example_img.squeeze(), cmap='gray')
plt.show()
